<a href="https://colab.research.google.com/github/amandatz/linear-programming/blob/main/Atividade1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 1



Amanda Topanotti Zanette (22100776)

**Importações e funções auxiliares**

In [6]:
using LinearAlgebra, Printf

Esse trecho de código é usado apenas para garantir a repetibilidade dos números aleatórios gerados.

In [7]:
using Random
Random.seed!(123)

TaskLocalRNG()

## Questão 1

**Gradiente com backtracking**

In [42]:
function steepest_descent_backtracking(fun, x0; sigma=1e-4, rho=0.5, max_iter=2000, tol=1e-6)
  x = copy(x0)

  for k=1:max_iter
    f, g, _ = fun(x)
    gnorm = norm(g)

    if gnorm <= tol
      return x, k, gnorm
    end

    # backtracking
    t = 1.0
    xn = x - t*g
    fn, _, _= fun(xn)

    while fn > f - sigma*t*gnorm^2
      t *= rho
      xn = x - t*g
      fn, _, _ = fun(xn)
    end

    x = xn
  end

  _, g, _ = fun(x)
  gnorm = norm(g)
  return x, max_iter, gnorm
end


steepest_descent_backtracking (generic function with 1 method)

**Newton com backtracking**

O método de Newton é dado por
$$
  x_{k+1} = x_k - \alpha_k H(x_k)^{-1} \nabla f(x_k)
$$
em que
- $\nabla f(x_k)$: gradiente de f;
- $H(x_k)$: matriz hessiana de f;
- $\alpha_k$: tamanho do passo.

In [45]:
function newton_backtracking(fun, x0; sigma=1e-4, rho=0.5, max_iter=2000, tol=1e-6)
  x = copy(x0)

  for k = 1:max_iter
    f, g, H = fun(x)
    gnorm = norm(g)

    if gnorm <= tol
      return x, k, gnorm
    end

    p = H \ g # VER ISSO DAQUI
    t = 1.0
    xn = x - t*p

    # backtracking
    while true
      fn, _, _ = fun(xn)

      if fn <= f - sigma*t*(g' * p)
        break
      end

      t *= rho
      xn = x - t*p
    end

    x = xn
  end

  f, g, H = fun(x)
  return x, max_iter, norm(g)
end


newton_backtracking (generic function with 1 method)

## Questão 2

Considere a função de Rosenbrock

In [49]:
function rosenb(x)
  a = 1.0
  b = 10.0

  f = (a - x[1])^2 + b*(x[2] - x[1]^2)^2

  grad_f = [ -2*(a - x[1]) + 2*b*(x[2] - x[1]^2)*(-2*x[1]),
        2*b*(x[2] - x[1]^2)]

  hess_f = [
        2 - 4*b*(x[2] - x[1]^2) + 8*b*x[1]^2   -4*b*x[1];
        -4*b*x[1]                               2*b
  ]

  return f, grad_f, hess_f
end

rosenb (generic function with 1 method)

Comparemos ambos os métodos anteriores considerando os pontos iniciais $x_0 = (1.01, 1.01)$ e $x_1 = (-1.0, 1.2)$

In [50]:
x0 = [1.01, 1.01]

_, max_iter_sd, _ = steepest_descent_backtracking(rosenb, x0)
println("Gradiente: ", max_iter_sd)

_, max_iter_n, _ = newton_backtracking(rosenb, x0)
println("Método de Newton: ", max_iter_n)


Gradiente: 1028
Método de Newton: 4


In [3]:
x1 = [-1.0, 1.2]

_, max_iter_sd, _ = steepest_descent_backtracking(rosenb, x1)
println("Gradiente: ", max_iter_sd)

_, max_iter_n, _ = newton_backtracking(rosenb, x1, max_iter=2000)
println("Método de Newton: ", max_iter_n) # VER ISSO DAQUI

LoadError: UndefVarError: `steepest_descent_backtracking` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## Questão 3

In [ ]:
function exact_line_search_quadratic(A, b, x, d)
    numerator = dot(b, d) - dot(x, A * d)
    denominator = dot(d, A * d)

    # retornar se a direção é muito pequena ou denominador proximo de zero
    if abs(denominator) < 1e-14 || norm(d) < 1e-12
        return 1e-6
    end

    return numerator / denominator
end

In [2]:
function quadratic(x, A)
  f = 0.5 * dot(x, A * x) - dot(b, x)
  g = A * x - b
  H = A
  return f, g, H
end

quadratica (generic function with 1 method)

In [4]:
function steepest_descent_exact(A, b, x0; max_iter=2000, tol=1e-12)
  x = copy(x0)

  f_evals = 0
  g_evals = 0

  for k = 1:max_iter
    f, g, _ = quadratic(x)
    g_evals += 1
    f_evals += 1

    gnorm = norm(g)

    if gnorm <= tol
      return x, k, gnorm, f_evals, g_evals
    end

    d = -g      # direção do gradiente

    # Busca linear exata
    alpha = exact_line_search_quadratic(A, b, x, d)

    # Atualização
    x = x + alpha * d
  end

  f, g, _ = quad_fun(x)
  g_evals += 1
  f_evals += 1
  gnorm = norm(g)
  return x, max_iter, gnorm, f_evals, g_evals
end

steepest_descent_exact (generic function with 1 method)

In [ ]:
dimensions = [10, 100, 1000]
cases = [:equal, :two, :spread]
